In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pickle

In [17]:
file_path = 'yelp_test_binary.txt'

# read data 
df = pd.read_csv(file_path, sep='\t', header=None, names=['Message', 'Gender', 'Confidence'])

# the number of rows
num_messages = df.shape[0]
print(f'Total number of messages in the dataset: {num_messages}')

Total number of messages in the dataset: 534460


In [18]:
df.head()

,Message,Gender,Confidence
0,i will never order flowers through cactus flow...,ABOUT:female,0.6285
1,ok but not special .,ABOUT:male,0.5086
2,where to even start ?,ABOUT:male,0.5796
3,my wife ordered the tenderloin sliders .,ABOUT:female,0.5699
4,"all so intelligent , pleasant and honest .",ABOUT:female,0.5788


In [19]:
# preprocessing on text data
df['Message'] = df['Message'].str.lower()
df['Message'] = df['Message'].str.replace('[^\w\s]', '')

In [20]:
# Encode the 'Gender' labels
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

# Tokenize and pad sequences
max_words = 1000  # can djust 
tokenizer = Tokenizer(num_words=max_words, split=' ')
tokenizer.fit_on_texts(df['Message'].values)
X = tokenizer.texts_to_sequences(df['Message'].values)
X = pad_sequences(X)

In [21]:
# target variable
y = df['Gender']

#  training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Build the LSTM model
embedding_dim = 50 
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=X.shape[1]))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [23]:
# Train the model
trained_model = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/5


6681/6681 [==============================] - 349s 52ms/step - loss: 0.4745 - accuracy: 0.7730 - val_loss: 0.4511 - val_accuracy: 0.7871
Epoch 2/5
6681/6681 [==============================] - 326s 49ms/step - loss: 0.4458 - accuracy: 0.7906 - val_loss: 0.4419 - val_accuracy: 0.7930
Epoch 3/5
6681/6681 [==============================] - 333s 50ms/step - loss: 0.4367 - accuracy: 0.7947 - val_loss: 0.4373 - val_accuracy: 0.7944
Epoch 4/5
6681/6681 [==============================] - 364s 54ms/step - loss: 0.4292 - accuracy: 0.7992 - val_loss: 0.4361 - val_accuracy: 0.7952
Epoch 5/5
6681/6681 [==============================] - 370s 55ms/step - loss: 0.4223 - accuracy: 0.8026 - val_loss: 0.4334 - val_accuracy: 0.7969


In [24]:
# Save model architecture and weights
model.save("lstm_trained_model.h5")

c:\Users\benur\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
# Save tokenizer
with open("lstm_tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
# Save label encoder
with open("lstm_label_encoder.pickle", "wb") as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
# Save max_length
with open("max_length_lstm.pickle", "w") as file:
    file.write(str(X.shape[1]))

In [35]:
# Save max length
with open("max_length.pickle", "wb") as handle:
    pickle.dump(max_length, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

3341/3341 [==============================] - 46s 14ms/step - loss: 0.4334 - accuracy: 0.7969
Accuracy: 0.7969071865081787


In [42]:
# Example
new_data = ['''''']
new_data_sequence = tokenizer.texts_to_sequences(new_data)
new_data_padded = pad_sequences(new_data_sequence, maxlen=X.shape[1])
prediction = model.predict(new_data_padded)
predicted_class = (prediction > 0.5).astype('int')[0][0]
predicted_gender = label_encoder.classes_[predicted_class]
print(f'Predicted Gender: {predicted_gender}')

1/1 [==============================] - 0s 20ms/step
Predicted Gender: ABOUT:male


: 